In [5]:
from casadi import *
import numpy as np
import os
import sys
sys.path.append("../")
# Linear equality constraints
A = np.concatenate([np.eye(4), np.eye(4)], axis=1)
b = np.ones((4,1))

x = MX.sym("x", 8)
z = MX.sym("z", x.shape)

f = sum((z-x) ** 2)
h = A @ z - b
nu = MX.sym("nu", h.shape)


# KKT matrix G
G = horzcat(jacobian(f, z) + nu.T @ jacobian(h, z), h.T).T

newton = rootfinder("newton", "newton", {"x": vertcat(z, nu), "g": G, "p": x})
z0 = DM(np.random.rand(12))
x_p = np.random.rand(8)

print("True: ", x)
z_res = newton(z0, x)
print("Constraints: ", A @ z[:8])
print(z_res)

True:  x
Constraints:  mac(
[[1, 0, 0, 0, 1, 0, 0, 0], 
 [0, 1, 0, 0, 0, 1, 0, 0], 
 [0, 0, 1, 0, 0, 0, 1, 0], 
 [0, 0, 0, 1, 0, 0, 0, 1]],z,zeros(4x1))
newton([0.251135, 0.677183, 0.846197, 0.401022, 0.46666, 0.335792, 0.464321, 0.718105, 0.690173, 0.46598, 0.5367, 0.715595], x){0}


In [2]:
nlp = {
    "f": f, "g": h, "x": z, "p": x
}
opts = {"ipopt.linear_solver": "ma27"}
solver = nlpsol("solver", "ipopt", nlp, opts)
sol = solver(x0=z0[:8], lbg=0, ubg=0, p=x_p)
display(sol["x"])

Exception of type: DYNAMIC_LIBRARY_FAILURE in file "Common/IpLibraryLoader.cpp" at line 72:
 Exception message: dlopen(libhsl.dylib, 0x0002): tried: 'libhsl.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OSlibhsl.dylib' (no such file), '/Users/jandavidridder/anaconda3/envs/apc/lib/python3.12/site-packages/casadi/libhsl.dylib' (no such file), '/Users/jandavidridder/anaconda3/envs/apc/bin/../lib/libhsl.dylib' (no such file), '/usr/lib/libhsl.dylib' (no such file, not in dyld cache), 'libhsl.dylib' (no such file), '/usr/local/lib/libhsl.dylib' (no such file), '/usr/lib/libhsl.dylib' (no such file, not in dyld cache)

EXIT: Library loading failure.
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       total  | 803.00us (803.00us)   1.11ms (  1.11ms)         1


DM([0.426644, 0.538658, 0.370501, 0.717906, 0.852609, 0.208643, 0.515973, 0.822134])

In [ ]:
from neurals.casadi import make_opt_layer

